In [2]:
import pandas as pd

data = pd.read_csv('data/SRP053101/SRP053101.tsv', sep='\t')

num_rows, num_cols = data.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

print(data.head())

Number of rows: 43355
Number of columns: 89
              Gene  SRR1785238  SRR1785239  SRR1785240  SRR1785241  \
0  ENSG00000000003    6.328873    6.213042    6.566083    7.431347   
1  ENSG00000000005   13.173146   12.876556   10.828189   12.335142   
2  ENSG00000000419    3.370290    3.527564    3.562937    3.661824   
3  ENSG00000000457    2.688080    2.836431    2.768769    2.896969   
4  ENSG00000000460    1.905655    1.892335    1.957807    1.944157   

   SRR1785242  SRR1785243  SRR1785244  SRR1785245  SRR1785246  ...  \
0    6.577885    6.622472    6.061065    6.185052    7.083283  ...   
1   22.767086   21.927400    3.438301    3.365119   17.128099  ...   
2    3.778322    3.697747    3.223593    3.277145    3.827415  ...   
3    3.017560    2.767193    2.720313    2.659437    2.979878  ...   
4    2.021323    1.936682    1.602395    1.960482    2.087508  ...   

   SRR1785316  SRR1785317  SRR1785318  SRR1785319  SRR1785320  SRR1785321  \
0    8.095718    6.850729    6.190149

In [3]:
# Load metadata
metadata = pd.read_csv('data/SRP053101/metadata_SRP053101.tsv', sep='\t')

# Check the unique values in refinebio_title to identify groups
print("Unique values in refinebio_title:")
print(metadata['refinebio_title'].unique())

# Display first few rows of metadata
print("\nFirst 5 rows of metadata:")
print(metadata[['refinebio_accession_code', 'refinebio_title', 'refinebio_treatment']].head())

# Count samples per group
print("\nSample counts per group:")
print(metadata['refinebio_title'].value_counts())

Unique values in refinebio_title:
['SAT01_T0' 'SAT02_T0' 'SAT03_T0' 'SAT04_T0' 'SAT05_T0' 'SAT06_T0'
 'SAT07_T0' 'SAT08_T0' 'SAT09_T0' 'SAT10_T0' 'SAT11_T0' 'SAT12_T0'
 'SAT13_T0' 'SAT14_T0' 'SAT15_T0' 'SAT16_T0' 'SAT17_T0' 'SAT18_T0'
 'SAT19_T0' 'SAT20_T0' 'SAT21_T0' 'SAT22_T0' 'SAT01_T3' 'SAT02_T3'
 'SAT03_T3' 'SAT04_T3' 'SAT05_T3' 'SAT06_T3' 'SAT07_T3' 'SAT08_T3'
 'SAT09_T3' 'SAT10_T3' 'SAT11_T3' 'SAT12_T3' 'SAT13_T3' 'SAT14_T3'
 'SAT15_T3' 'SAT16_T3' 'SAT17_T3' 'SAT18_T3' 'SAT19_T3' 'SAT20_T3'
 'SAT21_T3' 'SAT22_T3']

First 5 rows of metadata:
  refinebio_accession_code refinebio_title  refinebio_treatment
0               SRR1785238        SAT01_T0                  NaN
1               SRR1785239        SAT01_T0                  NaN
2               SRR1785240        SAT02_T0                  NaN
3               SRR1785241        SAT02_T0                  NaN
4               SRR1785242        SAT03_T0                  NaN

Sample counts per group:
refinebio_title
SAT01_T0    2
SAT02_

In [4]:
# Create group labels based on the timepoint (T0 vs T3)
metadata['group'] = metadata['refinebio_title'].apply(lambda x: 'T0' if x.endswith('_T0') else 'T3')

print("Group distribution:")
print(metadata['group'].value_counts())

# Map sample IDs to groups
sample_groups = dict(zip(metadata['refinebio_accession_code'], metadata['group']))

# Check which samples are in the expression data
expression_samples = data.columns[1:]  # Skip the first column (gene names)
print(f"\nExpression data has {len(expression_samples)} samples")
print(f"Metadata has {len(metadata)} samples")

# Filter metadata to match expression data samples
metadata_filtered = metadata[metadata['refinebio_accession_code'].isin(expression_samples)]
print(f"Matched samples: {len(metadata_filtered)}")

print("\nGroup distribution in matched samples:")
print(metadata_filtered['group'].value_counts())

Group distribution:
group
T0    44
T3    44
Name: count, dtype: int64

Expression data has 88 samples
Metadata has 88 samples
Matched samples: 88

Group distribution in matched samples:
group
T0    44
T3    44
Name: count, dtype: int64


In [5]:
import numpy as np
from scipy import stats
from bioinfokit.visuz import GeneExpression

# Prepare expression matrix for analysis
# Set gene names as index
expression_data = data.set_index(data.columns[0])
print(f"Expression data shape: {expression_data.shape}")

# Get T0 and T3 sample lists
t0_samples = metadata_filtered[metadata_filtered['group'] == 'T0']['refinebio_accession_code'].tolist()
t3_samples = metadata_filtered[metadata_filtered['group'] == 'T3']['refinebio_accession_code'].tolist()

print(f"T0 samples: {len(t0_samples)}")
print(f"T3 samples: {len(t3_samples)}")

# Separate expression data by groups
t0_expression = expression_data[t0_samples]
t3_expression = expression_data[t3_samples]

print("Expression data prepared for differential analysis")

Expression data shape: (43355, 88)
T0 samples: 44
T3 samples: 44
Expression data prepared for differential analysis


In [6]:
# Perform differential expression analysis
results = []

print("Performing differential expression analysis...")

for gene in expression_data.index:
    # Get expression values for each group
    t0_values = t0_expression.loc[gene].values
    t3_values = t3_expression.loc[gene].values
    
    # Calculate means
    t0_mean = np.mean(t0_values)
    t3_mean = np.mean(t3_values)
    
    # Calculate log2 fold change (T3 vs T0)
    # Add small pseudocount to avoid log(0)
    log2fc = np.log2((t3_mean + 1) / (t0_mean + 1))
    
    # Perform t-test
    t_stat, p_value = stats.ttest_ind(t3_values, t0_values)
    
    results.append({
        'gene': gene,
        'log2fc': log2fc,
        'pvalue': p_value,
        't0_mean': t0_mean,
        't3_mean': t3_mean,
        't_stat': t_stat
    })

# Convert to DataFrame
diff_results = pd.DataFrame(results)

# Calculate -log10(p-value) for volcano plot
diff_results['neg_log10_pval'] = -np.log10(diff_results['pvalue'])

print(f"Differential expression analysis completed for {len(diff_results)} genes")
print(f"Results shape: {diff_results.shape}")
print("\nFirst few results:")
print(diff_results.head())

Performing differential expression analysis...


/Users/I549593/miniforge3/envs/mmds/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


Differential expression analysis completed for 43355 genes
Results shape: (43355, 7)

First few results:
              gene    log2fc    pvalue    t0_mean   t3_mean    t_stat  \
0  ENSG00000000003  0.064623  0.052653   6.744615  7.099405  1.964909   
1  ENSG00000000005 -0.405473  0.000411  12.635671  9.294780 -3.676740   
2  ENSG00000000419 -0.028674  0.078483   3.666804  3.574967 -1.780761   
3  ENSG00000000457 -0.011067  0.434619   2.852458  2.823018 -0.784987   
4  ENSG00000000460 -0.011728  0.329757   1.955331  1.931405 -0.980158   

   neg_log10_pval  
0        1.278577  
1        3.386671  
2        1.105227  
3        0.361892  
4        0.481806  


In [7]:
# Summary statistics
print("=== Differential Expression Analysis Summary ===")
print(f"Total genes analyzed: {len(diff_results)}")

# Define significance thresholds
p_threshold = 0.05
log2fc_threshold = 1.0

# Count significant genes
significant_genes = diff_results[diff_results['pvalue'] < p_threshold]
print(f"Genes with p-value < {p_threshold}: {len(significant_genes)}")

upregulated = significant_genes[significant_genes['log2fc'] > log2fc_threshold]
downregulated = significant_genes[significant_genes['log2fc'] < -log2fc_threshold]

print(f"Significantly upregulated genes (log2FC > {log2fc_threshold}, p < {p_threshold}): {len(upregulated)}")
print(f"Significantly downregulated genes (log2FC < -{log2fc_threshold}, p < {p_threshold}): {len(downregulated)}")

# Range of fold changes and p-values
print(f"\nLog2 fold change range: {diff_results['log2fc'].min():.3f} to {diff_results['log2fc'].max():.3f}")
print(f"P-value range: {diff_results['pvalue'].min():.2e} to {diff_results['pvalue'].max():.3f}")
print(f"Most significant p-value: {diff_results['pvalue'].min():.2e}")

# Show top upregulated and downregulated genes
print(f"\nTop 5 upregulated genes:")
top_up = diff_results.nlargest(5, 'log2fc')[['gene', 'log2fc', 'pvalue']]
print(top_up)

print(f"\nTop 5 downregulated genes:")
top_down = diff_results.nsmallest(5, 'log2fc')[['gene', 'log2fc', 'pvalue']]
print(top_down)

=== Differential Expression Analysis Summary ===
Total genes analyzed: 43355
Genes with p-value < 0.05: 6213
Significantly upregulated genes (log2FC > 1.0, p < 0.05): 0
Significantly downregulated genes (log2FC < -1.0, p < 0.05): 1

Log2 fold change range: -1.035 to 1.099
P-value range: 2.54e-16 to 1.000
Most significant p-value: 2.54e-16

Top 5 upregulated genes:
                  gene    log2fc    pvalue
10501  ENSG00000161634  1.098763  0.131273
3755   ENSG00000110484  0.721314  0.091732
13798  ENSG00000176194  0.563053  0.000004
7893   ENSG00000140465  0.562992  0.161293
6167   ENSG00000130208  0.504207  0.050578

Top 5 downregulated genes:
                 gene    log2fc        pvalue
8618  ENSG00000145423 -1.034969  3.026298e-07
2904  ENSG00000104435 -0.998961  1.150985e-04
6893  ENSG00000134824 -0.910500  7.004280e-07
2557  ENSG00000101825 -0.893223  7.685373e-07
3606  ENSG00000109107 -0.756066  3.145691e-08


In [10]:
# Prepare data for volcano plot
# bioinfokit expects specific column names
volcano_data = diff_results.copy()
volcano_data = volcano_data.rename(columns={
    'gene': 'gene_name',
    'log2fc': 'log2FC',
    'pvalue': 'p_value'
})

# Remove any infinite or NaN values
volcano_data = volcano_data.replace([np.inf, -np.inf], np.nan).dropna()

print("Creating volcano plot...")
print(f"Data shape for volcano plot: {volcano_data.shape}")

# Create volcano plot with basic parameters
GeneExpression.volcano(
    df=volcano_data,
    lfc='log2FC',
    pv='p_value',
    genenames='gene_name',
    lfc_thr=(0.5, 0.5),  # log2FC thresholds
    pv_thr=(0.05, 0.05),  # p-value thresholds
    figname='volcano_plot_bariatric_rna_seq'
)

print("Volcano plot created and saved as 'volcano_plot_bariatric_rna_seq.png'")

Creating volcano plot...
Data shape for volcano plot: (43355, 7)


/Users/I549593/miniforge3/envs/mmds/lib/python3.11/site-packages/bioinfokit/visuz.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['color_add_axy'].fillna(color[1], inplace=True)  # intermediate


Volcano plot created and saved as 'volcano_plot_bariatric_rna_seq.png'


In [11]:
# Save differential expression results
output_file = 'differential_expression_results_T3_vs_T0.csv'
diff_results.to_csv(output_file, index=False)
print(f"Differential expression results saved to: {output_file}")

# Display final summary
print("\n" + "="*50)
print("DIFFERENTIAL EXPRESSION ANALYSIS SUMMARY")
print("="*50)
print(f"Comparison: T3 (3-month post-surgery) vs T0 (baseline)")
print(f"Total genes analyzed: {len(diff_results):,}")
print(f"Sample size: {len(t0_samples)} T0 samples, {len(t3_samples)} T3 samples")
print(f"")
print(f"Significance criteria:")
print(f"  - P-value threshold: < 0.05")
print(f"  - Log2 fold change threshold: |log2FC| > 0.5")
print(f"")
print(f"Results:")
print(f"  - Genes with p-value < 0.05: {len(significant_genes):,}")
print(f"  - Upregulated genes (log2FC > 0.5, p < 0.05): {len(upregulated):,}")
print(f"  - Downregulated genes (log2FC < -0.5, p < 0.05): {len(downregulated):,}")
print(f"")
print(f"Most significant genes:")
if len(significant_genes) > 0:
    top_sig = significant_genes.nsmallest(3, 'pvalue')[['gene', 'log2fc', 'pvalue']]
    for idx, row in top_sig.iterrows():
        direction = "↑" if row['log2fc'] > 0 else "↓"
        print(f"  {direction} {row['gene']}: log2FC = {row['log2fc']:.3f}, p = {row['pvalue']:.2e}")
print(f"")
print(f"Output files:")
print(f"  - Volcano plot: volcano_plot_bariatric_rna_seq.png")
print(f"  - Results table: {output_file}")
print("="*50)

Differential expression results saved to: differential_expression_results_T3_vs_T0.csv

DIFFERENTIAL EXPRESSION ANALYSIS SUMMARY
Comparison: T3 (3-month post-surgery) vs T0 (baseline)
Total genes analyzed: 43,355
Sample size: 44 T0 samples, 44 T3 samples

Significance criteria:
  - P-value threshold: < 0.05
  - Log2 fold change threshold: |log2FC| > 0.5

Results:
  - Genes with p-value < 0.05: 6,213
  - Upregulated genes (log2FC > 0.5, p < 0.05): 0
  - Downregulated genes (log2FC < -0.5, p < 0.05): 1

Most significant genes:
  ↑ ENSG00000135486: log2FC = 0.256, p = 2.54e-16
  ↑ ENSG00000198755: log2FC = 0.204, p = 5.98e-14
  ↓ ENSG00000113083: log2FC = -0.575, p = 6.02e-13

Output files:
  - Volcano plot: volcano_plot_bariatric_rna_seq.png
  - Results table: differential_expression_results_T3_vs_T0.csv
